In [1]:
from google.colab import drive
drive.mount('gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at gdrive


In [0]:
import pandas as pd
data = pd.read_csv('/content/gdrive/My Drive/final/b.csv')

In [0]:
!pip install transformers

     |████████████████████████████████| 573kB 15.2MB/s 
     |████████████████████████████████| 1.0MB 52.9MB/s 
     |████████████████████████████████| 3.7MB 50.1MB/s 
     |████████████████████████████████| 890kB 51.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=252f97a088d67d3a021846f261c2d886aef9ab516058f8853192fd787fbe27c0
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

Using TensorFlow backend.


In [0]:
data['bert']=0

In [0]:
data

,id,title,ingredients,description,step,total,hot,bert
0,6927971,매운 팽이버섯 조림 간단 반찬술안주맥주안주소주안주 굿굿,팽이버섯 고추장 진간장 설탕 다진마늘 물,밥반찬으로도 술안주로도 좋은 매운 팽이버섯 초림,"['팽이버섯 밑둥을 자르고 한입 크기 덩어리로 분리해줍니다', '팬에 기름을 두르고...",매운 팽이버섯 조림 간단 반찬술안주맥주안주소주안주 굿굿 팽이버섯 고추장 진간장 설탕...,1,0
1,6867650,다른 반찬이 필요없는 매운 어묵 비빔밥,사각어묵 땡초 통마늘 고추장 굴소스 올리고당 고춧가루 후추 통깨 올리브유 맛술 참기름,다른 반찬이 필요없는 어묵 양념 비빔밥 재료 착한 가격에 손쉽게 만들어 먹을수 있...,"['어묵 ᆞ땡초 다져주시고 통마늘 듬성듬성 다져주세요', '팬에 위 준비한 재료를 ...",다른 반찬이 필요없는 매운 어묵 비빔밥 사각어묵 땡초 통마늘 고추장 굴소스 올리고당...,1,0
2,6860267,간단 자취요리 미친 매운맛 빨간오뎅 집에서 만들어 먹기 Spicy fish cakes,어묵 물 고추 고추장 간장 설탕 다진마늘,안녕하세요 여러분 오늘은 빨간오뎅을 만들어 봤어요 빨간 어묵이 맞지만 상호명이기에 ...,"['재료를 준비합니다어묵 물 고추 고추장 간장 설탕 다진마늘', '고추 1개는 썰고...",간단 자취요리 미친 매운맛 빨간오뎅 집에서 만들어 먹기 Spicy fish cak...,1,0
3,6910384,매운맛 쏙 마늘 꿀조림 만들기,간장 꿀 맛술 마늘,마늘은 매운맛때문에 먹기힘들죠 그런데 이렇게 하니 아이들도 먹을수있고 마늘이 꿀맛...,"['재료 마늘2컵 꿀3큰술맛술1큰술조림장 간장2큰술꿀3큰술맛술1큰술', '마늘은 껍...",매운맛 쏙 마늘 꿀조림 만들기 간장 꿀 맛술 마늘 마늘은 매운맛때문에 먹기힘들죠 ...,1,0
4,6883669,매운콩나물잡채,당면 콩나물 양파 당근 대파 물 참기름 통깨 고추장 고춧가루 간장 설탕 다진 마늘 ...,콩나물잡채,['당면은 찬물에 밀 불려서 준비해 둡니다대략 30분 이상 불려주심 된답니다당면 전...,매운콩나물잡채 당면 콩나물 양파 당근 대파 물 참기름 통깨 고추장 고춧가루 간장 ...,1,0
...,...,...,...,...,...,...,...,...
91553,6844474,부드럽고 달콤한 디저트 우유 생크림 젤리 만드는 법,우유 생크림 설탕 젤라틴 분말 초코시럽 딸기시럽 럼주 딸기시럽 초코시럽 물 젤라틴 ...,고소하고 부드러운 우유와 생크림에 쌉쌀한 초코와 상큼한 딸기가 환상의 디저트를 만들...,['냄비에 우유 생크림 설탕 젤라틴 분말을 넣고 중불에서 젤라틴이 녹을 정도로 끓여...,부드럽고 달콤한 디저트 우유 생크림 젤리 만드는 법 우유 생크림 설탕 젤라틴 분말 ...,0,0
91554,6844473,오리훈제 야채샐러드,오리훈제슬라이스 쌈배추 파프리카 쌀뜨물 사과 양배추 간장 들깨가루 들기름 레몬즙 올...,야채가 더 많이 들어가서 위에 부담도 적고 오리훈제 야채샐러드 한접시면 양양가득 이...,['오리훈제 야채샐러드 재료 알아봅니다 오리훈제슬라이스 20조각 쌈배추 3장 파프리...,오리훈제 야채샐러드 오리훈제슬라이스 쌈배추 파프리카 쌀뜨물 사과 양배추 간장 들깨가...,0,0
91555,6844471,오징어 굴 소스 야채 볶음밥,오징어 콩나물 양배추 파 마늘 양파 당근 브로콜리 파프리카 밥 식용유 들기름 굴소소,딸 친구가 갑자기 놀러 왔어요 밥통에 밥은 있는데 반찬이 영 그래서 냉동실에 손질해...,"['야채 잔뜩 썰어 놨어요 숙주 없어 콩나물 준비했어요', '밥은 2공기 준비하세요...",오징어 굴 소스 야채 볶음밥 오징어 콩나물 양배추 파 마늘 양파 당근 브로콜리 파프...,2,0
91556,6844470,깨쟁이요리최고의브런치시금치 바질 페스토시금치 바질 파스타 스파게티,바질페스토 스파게티면 양파 채썬마늘 닭가슴살 생크림.우유 파마산치즈 바질페스토,집에서 직접키운 바질로 만든 바질페스토 시금치를 곁들여 만든 시금치 바질페스토 파스...,['바질페스토시금치 14단바질2팩올리브유2컵잣12컵마늘2알파마산치즈3Ts앤초비1마리...,깨쟁이요리최고의브런치시금치 바질 페스토시금치 바질 파스타 스파게티 바질페스토 스파게...,2,0


In [0]:
data['step'][0]

"['팽이버섯 밑둥을 자르고 한입 크기 덩어리로 분리해줍니다', '팬에 기름을 두르고 팽이버섯을 앞뒤로 30초씩 구워 줍니다강불', '분량의 소스를 팽이버섯 위에 부어 줍니다', '앞뒤로 30초씩 조려 주면 완성']"

In [0]:
test1 = data[data['hot']==1]
test2 = data[data['hot']==0]
test=pd.concat([test1,test2])

In [0]:
test=test.reset_index()

In [0]:
test

,index,id,title,ingredients,description,step,total,hot,bert
0,0,6927971,매운 팽이버섯 조림 간단 반찬술안주맥주안주소주안주 굿굿,팽이버섯 고추장 진간장 설탕 다진마늘 물,밥반찬으로도 술안주로도 좋은 매운 팽이버섯 초림,"['팽이버섯 밑둥을 자르고 한입 크기 덩어리로 분리해줍니다', '팬에 기름을 두르고...",매운 팽이버섯 조림 간단 반찬술안주맥주안주소주안주 굿굿 팽이버섯 고추장 진간장 설탕...,1,0
1,1,6867650,다른 반찬이 필요없는 매운 어묵 비빔밥,사각어묵 땡초 통마늘 고추장 굴소스 올리고당 고춧가루 후추 통깨 올리브유 맛술 참기름,다른 반찬이 필요없는 어묵 양념 비빔밥 재료 착한 가격에 손쉽게 만들어 먹을수 있...,"['어묵 ᆞ땡초 다져주시고 통마늘 듬성듬성 다져주세요', '팬에 위 준비한 재료를 ...",다른 반찬이 필요없는 매운 어묵 비빔밥 사각어묵 땡초 통마늘 고추장 굴소스 올리고당...,1,0
2,2,6860267,간단 자취요리 미친 매운맛 빨간오뎅 집에서 만들어 먹기 Spicy fish cakes,어묵 물 고추 고추장 간장 설탕 다진마늘,안녕하세요 여러분 오늘은 빨간오뎅을 만들어 봤어요 빨간 어묵이 맞지만 상호명이기에 ...,"['재료를 준비합니다어묵 물 고추 고추장 간장 설탕 다진마늘', '고추 1개는 썰고...",간단 자취요리 미친 매운맛 빨간오뎅 집에서 만들어 먹기 Spicy fish cak...,1,0
3,3,6910384,매운맛 쏙 마늘 꿀조림 만들기,간장 꿀 맛술 마늘,마늘은 매운맛때문에 먹기힘들죠 그런데 이렇게 하니 아이들도 먹을수있고 마늘이 꿀맛...,"['재료 마늘2컵 꿀3큰술맛술1큰술조림장 간장2큰술꿀3큰술맛술1큰술', '마늘은 껍...",매운맛 쏙 마늘 꿀조림 만들기 간장 꿀 맛술 마늘 마늘은 매운맛때문에 먹기힘들죠 ...,1,0
4,4,6883669,매운콩나물잡채,당면 콩나물 양파 당근 대파 물 참기름 통깨 고추장 고춧가루 간장 설탕 다진 마늘 ...,콩나물잡채,['당면은 찬물에 밀 불려서 준비해 둡니다대략 30분 이상 불려주심 된답니다당면 전...,매운콩나물잡채 당면 콩나물 양파 당근 대파 물 참기름 통깨 고추장 고춧가루 간장 ...,1,0
...,...,...,...,...,...,...,...,...,...
43846,91545,6844483,버섯리조또도오전,말린버섯 쌀 불린 것 버터 마늘 바질 치즈두장,쉬운 버섯을 활용한 색다른 요리 건강잡고 입맛 찾고,['말린버섯은 한 번 씻어주고 뜨거운 물에 불려요 물은 사용해야 하니 버리지 않습니...,버섯리조또도오전 말린버섯 쌀 불린 것 버터 마늘 바질 치즈두장 쉬운 버섯을 활용한...,0,0
43847,91550,6844477,달고 맛 있는 밤밥,밤 쌀 파 마늘 간장 깨소금 참기름,한국 공주에서온 밤으로 만든 밤밥 입니다,"['쌀밤 물 준비합니다', '밤을 껍질을 까서 쌀과같이 밥을 짓습니다', '양념장을...",달고 맛 있는 밤밥 밤 쌀 파 마늘 간장 깨소금 참기름 한국 공주에서온 밤으로 만...,0,0
43848,91552,6844475,노오븐 베이킹핫케잌 믹스로 만든 밥솥 케이크,핫케잌믹스 우유 계란 견과류 건과일 올리브유,베이킹 초보가 만들수 있는 밥솥 케이이크인데 핫케잌 믹스로 간단하게 만들어 봤어요...,"['재료 준비', '건자두와 견과류를 잘게 다져 주세요', '그릇에 계란한개를 깨 ...",노오븐 베이킹핫케잌 믹스로 만든 밥솥 케이크 핫케잌믹스 우유 계란 견과류 건과일 올...,0,0
43849,91553,6844474,부드럽고 달콤한 디저트 우유 생크림 젤리 만드는 법,우유 생크림 설탕 젤라틴 분말 초코시럽 딸기시럽 럼주 딸기시럽 초코시럽 물 젤라틴 ...,고소하고 부드러운 우유와 생크림에 쌉쌀한 초코와 상큼한 딸기가 환상의 디저트를 만들...,['냄비에 우유 생크림 설탕 젤라틴 분말을 넣고 중불에서 젤라틴이 녹을 정도로 끓여...,부드럽고 달콤한 디저트 우유 생크림 젤리 만드는 법 우유 생크림 설탕 젤라틴 분말 ...,0,0


In [0]:
test = test[0:10000]

In [0]:
from collections import Counter
Counter(test['hot']).items()

dict_items([(1, 3762), (0, 6238)])

In [0]:
test['step'][0]

"['팽이버섯 밑둥을 자르고 한입 크기 덩어리로 분리해줍니다', '팬에 기름을 두르고 팽이버섯을 앞뒤로 30초씩 구워 줍니다강불', '분량의 소스를 팽이버섯 위에 부어 줍니다', '앞뒤로 30초씩 조려 주면 완성']"

In [0]:
for i in range(0,10000):
    try:
      temp_step=test['step'][i].replace("[",'').replace("]",'').replace("'",'').replace(',',' [SEP]')
      temp= "[CLS]"+test['title'][i]+' '+test['ingredients'][i]+' '+test['description'][i]+' [SEP]'+temp_step
      test['bert'][i]=temp
    except:
        print(i)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:1024: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc[key] = value
/usr/local/lib/python3.6/dist-pac

In [0]:
data['bert0']=0
for i in range(0,91558):
    try:
      temp_step=data['step'][i].replace("[",'').replace("]",'').replace("'",'').replace(',',' [SEP]')
      temp= "[CLS]"+data['title'][i]+' '+data['description'][i]+' [SEP]'+temp_step
      data['bert0'][i]=temp
    except:
        print(i)

,index,id,title,ingredients,description,step,total,hot,bert
0,0,6927971,매운 팽이버섯 조림 간단 반찬술안주맥주안주소주안주 굿굿,팽이버섯 고추장 진간장 설탕 다진마늘 물,밥반찬으로도 술안주로도 좋은 매운 팽이버섯 초림,"['팽이버섯 밑둥을 자르고 한입 크기 덩어리로 분리해줍니다', '팬에 기름을 두르고...",매운 팽이버섯 조림 간단 반찬술안주맥주안주소주안주 굿굿 팽이버섯 고추장 진간장 설탕...,1,[CLS]매운 팽이버섯 조림 간단 반찬술안주맥주안주소주안주 굿굿 팽이버섯 고추장 진...
1,1,6867650,다른 반찬이 필요없는 매운 어묵 비빔밥,사각어묵 땡초 통마늘 고추장 굴소스 올리고당 고춧가루 후추 통깨 올리브유 맛술 참기름,다른 반찬이 필요없는 어묵 양념 비빔밥 재료 착한 가격에 손쉽게 만들어 먹을수 있...,"['어묵 ᆞ땡초 다져주시고 통마늘 듬성듬성 다져주세요', '팬에 위 준비한 재료를 ...",다른 반찬이 필요없는 매운 어묵 비빔밥 사각어묵 땡초 통마늘 고추장 굴소스 올리고당...,1,[CLS]다른 반찬이 필요없는 매운 어묵 비빔밥 사각어묵 땡초 통마늘 고추장 굴소스...
2,2,6860267,간단 자취요리 미친 매운맛 빨간오뎅 집에서 만들어 먹기 Spicy fish cakes,어묵 물 고추 고추장 간장 설탕 다진마늘,안녕하세요 여러분 오늘은 빨간오뎅을 만들어 봤어요 빨간 어묵이 맞지만 상호명이기에 ...,"['재료를 준비합니다어묵 물 고추 고추장 간장 설탕 다진마늘', '고추 1개는 썰고...",간단 자취요리 미친 매운맛 빨간오뎅 집에서 만들어 먹기 Spicy fish cak...,1,[CLS]간단 자취요리 미친 매운맛 빨간오뎅 집에서 만들어 먹기 Spicy fis...
3,3,6910384,매운맛 쏙 마늘 꿀조림 만들기,간장 꿀 맛술 마늘,마늘은 매운맛때문에 먹기힘들죠 그런데 이렇게 하니 아이들도 먹을수있고 마늘이 꿀맛...,"['재료 마늘2컵 꿀3큰술맛술1큰술조림장 간장2큰술꿀3큰술맛술1큰술', '마늘은 껍...",매운맛 쏙 마늘 꿀조림 만들기 간장 꿀 맛술 마늘 마늘은 매운맛때문에 먹기힘들죠 ...,1,[CLS]매운맛 쏙 마늘 꿀조림 만들기 간장 꿀 맛술 마늘 마늘은 매운맛때문에 먹...
4,4,6883669,매운콩나물잡채,당면 콩나물 양파 당근 대파 물 참기름 통깨 고추장 고춧가루 간장 설탕 다진 마늘 ...,콩나물잡채,['당면은 찬물에 밀 불려서 준비해 둡니다대략 30분 이상 불려주심 된답니다당면 전...,매운콩나물잡채 당면 콩나물 양파 당근 대파 물 참기름 통깨 고추장 고춧가루 간장 ...,1,[CLS]매운콩나물잡채 당면 콩나물 양파 당근 대파 물 참기름 통깨 고추장 고춧가...
...,...,...,...,...,...,...,...,...,...
9995,19067,6829636,향긋한 깻잎나물깻잎볶음,깻잎 다듬은 양 집간장 고운천일염 다진마늘 다진대파 들기름 통깨,한참 봄나물로 가득하던곳이 요즘은 깻잎이 자리를 차지하고 있어서 한다발 담아와 조물...,['신문지로 말아 꽃다발처럼 보기는 좋았는데요 요 줄기는 똑똑다듬어 주어야해요 다듬...,향긋한 깻잎나물깻잎볶음 깻잎 다듬은 양 집간장 고운천일염 다진마늘 다진대파 들기름 ...,0,[CLS]향긋한 깻잎나물깻잎볶음 깻잎 다듬은 양 집간장 고운천일염 다진마늘 다진대파...
9996,19069,6908659,미국식 핫도그 맛보장 꿀조합,핫도그 빵 소시지 로메인 칠리소스 캐첩 머스터드소스 치즈,맛이 없을 수 없는 꿀조합 오늘 네이버 푸드 메인에 뜬 쏭아의 미국식 핫도그 쉽고 ...,"['소시지에 칼집을 내고 팬에 살짝 익혀준다', '핫도그 빵 사이에 소시지를 넣고칠...",미국식 핫도그 맛보장 꿀조합 핫도그 빵 소시지 로메인 칠리소스 캐첩 머스터드소스 치...,0,[CLS]미국식 핫도그 맛보장 꿀조합 핫도그 빵 소시지 로메인 칠리소스 캐첩 머스터...
9997,19072,1224012,맛난 빠다코코넛,박력분 코코넛가루 뿌려줄설탕 덧가루용강력분 베이킹파우더 설탕 소금 달걀 포도씨유 버터,간식으로 사랑받는 빠다코코넛을 올려봅니다 근데 사각 주름 커터가 없어서 전혀다른 녀...,['실온의 버터를 크림화 해서 포도씨유를 넣고 잘 섞어줍니다 설탕을 넣고 잘 섞어줍...,맛난 빠다코코넛 박력분 코코넛가루 뿌려줄설탕 덧가루용강력분 베이킹파우더 설탕 소금 ...,0,[CLS]맛난 빠다코코넛 박력분 코코넛가루 뿌려줄설탕 덧가루용강력분 베이킹파우더 설...
9998,19073,6876077,가지 싫어하는 사람들 입맛 잡은 영양만점 가지밥,밥 큰가지 간돼지고기 맛술 간장 대파뿌리 식용유,간돼지고기와 가지를 볶아 가지밥의 맛을 살린 영양만점 레시피 ...,['총총 썬 대파뿌리부분를 먼저 식용유에 볶아 주세요대파가 노릇해지면 간돼지고기종이...,가지 싫어하는 사람들 입맛 잡은 영양만점 가지밥 밥 큰가지 간돼지고기 맛술 간장 ...,0,[CLS]가지 싫어하는 사람들 입맛 잡은 영양만점 가지밥 밥 큰가지 간돼지고기 맛...


In [0]:
test.to_csv("/content/gdrive/My Drive/final/c.csv", mode='w', index=False)

In [0]:
test_bart=test.sample(frac=1).reset_index(drop=True)

In [0]:
train=test_bart[0:7000]

In [0]:
test=test_bart[7000:10000]

In [0]:
sentences = [str(sentence) for sentence in train['bert']]
sentences[:10]

['[CLS]쉬운 계량으로 만드는 오레오 머랭쿠키 달걀흰자 설탕 오레오  어려운 계량이 아닌 쉬운 숟가락 계량법으로 만드는 쉽고 간단한 오레오 머랭쿠키 만들기                 [SEP]냉장보관한 달걀의 흰자를 볼에 담아주세요 [SEP] 핸드믹서로 흰자를 휘핑해주시고 흰자에 큰 거품이 올라오면 설탕 2숟갈을 넣고 휘핑해주세요 [SEP] 거품이 조밀해지면 다시 설탕 2숟갈을 넣어줍니다 [SEP] 12분정도 더 휘핑을 하다 마지막으로 설탕 2숟갈을 넣고 설탕입자가 보이지 않을 때까지 계속 휘핑해주세요 [SEP] 머랭에서 윤기가 흐르고 핸드믹서를 들어올렸을 때 독수리 부리모양의 뿔이 생기면 머랭 완성 [SEP] 짤주머니에 공기가 없도록 머랭을 채워주세요머랭 사이에 공간이 있으면 머랭을 짤 때 펑 하고 모양이 이상해져요 [SEP] 머랭 사이에 공간이 있으면 머랭에 구멍이 생겨버려요 [SEP] 오븐은 110°로 10분동안 예열해주시고 종이호일이나 테프론시트지에 머랭을 짜주세요테프론시트지는 종이호일보다 머랭이 더 예쁘고 깔끔하게 때어지는 장점이 있답니다 [SEP] 오레오를 샌드와 크림으로 분리하고 샌드를 비닐봉지에 넣어 부수거나 믹서기에 갈아주세요저는 오레오 7개를 갈았다가 많이 남아서 45개정도가 적당할 듯 하네요 [SEP] 오레오를 머랭 위에 뿌려주시고예열한 오븐에서 100°로 1시간 30분동안 구워줍니다 [SEP] 머랭이 완성되고 열기가 어느정도 빠져나가면 밀폐용기에 보관해주세요머랭쿠키가 공기에 접촉하면 찐득찐득해지기 때문에 머랭쿠키를 오븐에서 식혔다가 꺼내주세요 [SEP] 예쁘게 포장하면 완성',
 '[CLS]초콜릿 크림치즈 컵케이크 만들기｜효썸 크림치즈 무염버터 설탕 달걀 바닐라익스트랙 다크 초콜릿 무염버터 달걀 설탕 박력분 베이킹파우더 바닐라익스트랙  초콜릿과 버터를 사용해도 식감이 정말 가벼워요냉장고에 보관하면 조금 단단해지겠지만 그리고 맛도 정말 진해서 우유랑 먹으면 더 맛있습니다 5개 분량                 [SEP]크림치즈 반죽 볼

In [0]:
labels = train['hot'].values
labels

array([0, 0, 1, ..., 0, 1, 0])

In [0]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])


[CLS]쉬운 계량으로 만드는 오레오 머랭쿠키 달걀흰자 설탕 오레오  어려운 계량이 아닌 쉬운 숟가락 계량법으로 만드는 쉽고 간단한 오레오 머랭쿠키 만들기                 [SEP]냉장보관한 달걀의 흰자를 볼에 담아주세요 [SEP] 핸드믹서로 흰자를 휘핑해주시고 흰자에 큰 거품이 올라오면 설탕 2숟갈을 넣고 휘핑해주세요 [SEP] 거품이 조밀해지면 다시 설탕 2숟갈을 넣어줍니다 [SEP] 12분정도 더 휘핑을 하다 마지막으로 설탕 2숟갈을 넣고 설탕입자가 보이지 않을 때까지 계속 휘핑해주세요 [SEP] 머랭에서 윤기가 흐르고 핸드믹서를 들어올렸을 때 독수리 부리모양의 뿔이 생기면 머랭 완성 [SEP] 짤주머니에 공기가 없도록 머랭을 채워주세요머랭 사이에 공간이 있으면 머랭을 짤 때 펑 하고 모양이 이상해져요 [SEP] 머랭 사이에 공간이 있으면 머랭에 구멍이 생겨버려요 [SEP] 오븐은 110°로 10분동안 예열해주시고 종이호일이나 테프론시트지에 머랭을 짜주세요테프론시트지는 종이호일보다 머랭이 더 예쁘고 깔끔하게 때어지는 장점이 있답니다 [SEP] 오레오를 샌드와 크림으로 분리하고 샌드를 비닐봉지에 넣어 부수거나 믹서기에 갈아주세요저는 오레오 7개를 갈았다가 많이 남아서 45개정도가 적당할 듯 하네요 [SEP] 오레오를 머랭 위에 뿌려주시고예열한 오븐에서 100°로 1시간 30분동안 구워줍니다 [SEP] 머랭이 완성되고 열기가 어느정도 빠져나가면 밀폐용기에 보관해주세요머랭쿠키가 공기에 접촉하면 찐득찐득해지기 때문에 머랭쿠키를 오븐에서 식혔다가 꺼내주세요 [SEP] 예쁘게 포장하면 완성
['[CLS]', '쉬', '##운', '계', '##량', '##으로', '만', '##드는', '오', '##레', '##오', '머', '##랭', '##쿠', '##키', '달', '##걀', '##흰', '##자', '설', '##탕', '오', '##레', '##오', '어', '##려', '##운', '계', '##량', '##이', '아닌', '쉬',

In [0]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 512

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9469,  21614,   8887,  44321,  11467,   9248,  61945,
         9580,  56645,  28188,   9265, 118861,  61156,  21039,   9061,
       118621, 119468,  13764,   9429, 119341,   9580,  56645,  28188,
         9546,  26737,  21614,   8887,  44321,  10739,  63783,   9469,
        21614,    100,   8887,  44321,  33768,  11467,   9248,  61945,
         9471,  11664,   8845,  24989,  11102,   9580,  56645,  28188,
         9265, 118861,  61156,  21039,   9248,  27023,  12310,    102,
         9001,  13890,  30005,  20595,  11102,   9061, 118621,  10459,
        10024,  48959,   9359,  10530,   9064,  16985,  16323,  24982,
        48549,    102,   9962,  15001, 118956,  12424,  11261,  10024,
        48959,  10010, 119418,  14523,  87281,  11664,  10024,  13764,
        10530,   9835,   8863,  52951,  10739,   9583,  17342,  28188,
        14867,   9429, 119341,    100,   9010,  11664,  10010, 119418,
        14523,  16323,  24982,  48549,    102,   8863,  52951,  10739,
      

In [0]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [0]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,  10150,  37712,   9873,  34994,   9248,  27023,  12310,   9486,
        119010,   9061, 118621,   9344,  10739, 119297,   9283, 119235, 118874,
         17342,  18622,  24891,  10150,  37712,   9873,  34994,   9736,  25934,
        119373,  26444,  34994,   9248,  93200, 101450,  14153,  48549,    102,
          9911,  10530,   8932,  49543,  10622,   9678,  40032,   9102,  81220,
          9344,  10739, 119297,  10622,   9531,   9109,  11261,   9639, 119472,
         48345,   9639, 119469,   9344,  10739, 119297,  10622,   9838,  55358,
         22695,  38851,  11489,   8932,  49543,  10622,   9672,  41521,  11102,
         10003,   9654,  14153,   9505,  12965,    100,    102,   9486, 119010,
         10622,   9736,  25934, 119373,  92383,   9011,  39420,  16439,  14153,
          9580,  26737,    100,    102,   9911,  10530,   9336,  21876,  34959,
         10240,  10622,   9010,  11664,   9486, 119010,  10622,   9029, 119081,
         48345,   9368,  10892,   9539, 

In [0]:
# 배치 사이즈
batch_size = 4

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
# 리뷰 문장 추출
sentences = [str(sentence) for sentence in test['bert']]
sentences[:10]

['[CLS]NO오븐 NO밀가루 NO설탕 NO버터 오직 단호박만으로 만든 다이어트빵 단호박빵 단호박 계란 견과류 꿀  안녕하세요 마이쏭이에여 다이어트할때 면과 빵의 유혹이 어찌나 힘든지 다이어터라면 공감 100  아시쥬 빵 먹고 찐살은 지구 열두바퀴를 돌아도 안빠진다는 거 그래서 오늘은 NO오븐 NO밀가루 NO설탕 NO버터 오직 단호박만으로 만든 다이어트빵 단호박빵을 만들어볼꺼에여요거요거 전에 마이쏭이 고구마로 만든 고구마빵을 포스팅 했었는데 오늘은 단호박으로 만들었어요 빵에 견과류도 넣어 식감도 UP 시켰답니다  든든하고 살안찌는 야식으로 아이들 간식으로도 아침브런치로도 좋은 건강하고 속편한빵 만들어보자구욧                 [SEP]단호박은 반으로 갈라 수저로 씨를 파내고 찜기에 쪄주셔도 되고 간단하게 위생비닐에 넣고 공기가 통하게 살짝 묶어 전자렌지에 5분정도 돌려잘 익혀줍니다 [SEP] 그리고 익힌 단호박은 으깨주어요 [SEP] 팍팍 잘 으깨진 단호박이 준비 되셨다면 [SEP] 계란을 분리해 줄 차례흰자는 따로 좀 큰볼에 담아놓고 [SEP] 노른자는 으깬단호박과 잘 섞어줍니다 [SEP] 고구마로 하실땐 우유를 조금 부어 부드럽게 만들어주면 맛있지만단호박은 우유를 넣으면 질퍽해지는 느낌이라서 패쓰 [SEP] 요 흰자는 머랭을 쳐줄꺼에요 [SEP] 거품기가 있어 요걸로 쉽게 만들 [SEP] 어 주려고 했으나거품기가 너무작아서 그냥 손으로 하는게 훨 빠른듯 ㅋㅋㅋㅋㅋ [SEP] 한 방향으로 재빨리저어 요렇게 거품을 만들어주어요 [SEP] 특별한 스킬없어요 ㅋㅋㅋㅋ 그냥 계속 저어주시면 되요전혀 어렵지 않음 but 물이 들어가지않게 주의 [SEP] 거품이 줄줄 흐르지않게 생크림처럼 만들어주시면 되요그릇을 엎어도 팍 안떨어지게 [SEP] 요렇게 만든 머랭은 단호박에 잘 섞어주고 [SEP] 여기에 꿀을 한두큰술 넣어 준답니다 [SEP] 오늘은 단호박빵에 견과류도 넣어줄 예정견과류는 봉지를 뜯지않고 잘 빻아 다져줍니다 [SEP] 마이쏭은 한봉지 넣었는데 

In [0]:
# 라벨 추출
labels = test['hot'].values
labels

array([0, 0, 0, ..., 0, 1, 0])

In [0]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS]NO오븐 NO밀가루 NO설탕 NO버터 오직 단호박만으로 만든 다이어트빵 단호박빵 단호박 계란 견과류 꿀  안녕하세요 마이쏭이에여 다이어트할때 면과 빵의 유혹이 어찌나 힘든지 다이어터라면 공감 100  아시쥬 빵 먹고 찐살은 지구 열두바퀴를 돌아도 안빠진다는 거 그래서 오늘은 NO오븐 NO밀가루 NO설탕 NO버터 오직 단호박만으로 만든 다이어트빵 단호박빵을 만들어볼꺼에여요거요거 전에 마이쏭이 고구마로 만든 고구마빵을 포스팅 했었는데 오늘은 단호박으로 만들었어요 빵에 견과류도 넣어 식감도 UP 시켰답니다  든든하고 살안찌는 야식으로 아이들 간식으로도 아침브런치로도 좋은 건강하고 속편한빵 만들어보자구욧                 [SEP]단호박은 반으로 갈라 수저로 씨를 파내고 찜기에 쪄주셔도 되고 간단하게 위생비닐에 넣고 공기가 통하게 살짝 묶어 전자렌지에 5분정도 돌려잘 익혀줍니다 [SEP] 그리고 익힌 단호박은 으깨주어요 [SEP] 팍팍 잘 으깨진 단호박이 준비 되셨다면 [SEP] 계란을 분리해 줄 차례흰자는 따로 좀 큰볼에 담아놓고 [SEP] 노른자는 으깬단호박과 잘 섞어줍니다 [SEP] 고구마로 하실땐 우유를 조금 부어 부드럽게 만들어주면 맛있지만단호박은 우유를 넣으면 질퍽해지는 느낌이라서 패쓰 [SEP] 요 흰자는 머랭을 쳐줄꺼에요 [SEP] 거품기가 있어 요걸로 쉽게 만들 [SEP] 어 주려고 했으나거품기가 너무작아서 그냥 손으로 하는게 훨 빠른듯 ㅋㅋㅋㅋㅋ [SEP] 한 방향으로 재빨리저어 요렇게 거품을 만들어주어요 [SEP] 특별한 스킬없어요 ㅋㅋㅋㅋ 그냥 계속 저어주시면 되요전혀 어렵지 않음 but 물이 들어가지않게 주의 [SEP] 거품이 줄줄 흐르지않게 생크림처럼 만들어주시면 되요그릇을 엎어도 팍 안떨어지게 [SEP] 요렇게 만든 머랭은 단호박에 잘 섞어주고 [SEP] 여기에 꿀을 한두큰술 넣어 준답니다 [SEP] 오늘은 단호박빵에 견과류도 넣어줄 예정견과류는 봉지를 뜯지않고 잘 빻아 다져줍니다 [SEP] 마이쏭은 한봉지 넣었는데 만들

In [0]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 512

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,  49307,  28188, 119000,  49307, 118958,  11287,  35866,
        49307,  31928, 119341,  49307,  41605,  21876,   9580,  33077,
         9059,  20309, 118963,  19105,  11467,  73306,   9056,  86732,
        15184, 119010,   9059,  20309, 118963, 119010,   9059,  20309,
       118963,   8887,  49919,   8880,  11882,  46520,   8964,   9521,
       118741,  35506,  24982,  48549,    100,   9056,  86732,  15184,
        14843, 118832,   9279,  11882,   9390,  10459,   9625, 119438,
        10739,   9546, 119245,  16439,  10028,  90537,  12508,   9056,
        86732,  21876,  17342,  14867,   8896, 105197,  10407,   9519,
        14040, 119224,   9390,   9266,  11664,    100,   9706,  17196,
         9569, 118802,  42144, 119322,  11513,   9091,  16985,  12092,
         9521, 119008,  65096,  11018,   8863,   8924,  37388,  12424,
         9580, 118762,  10892,  49307,  28188, 119000,  49307, 118958,
        11287,  35866,  49307,  31928, 119341,  49307,  41605,  21876,
      

In [0]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [0]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,  49307,  28188, 119000,  49307, 118958,  11287,  35866,  49307,
         31928, 119341,  49307,  41605,  21876,   9580,  33077,   9059,  20309,
        118963,  19105,  11467,  73306,   9056,  86732,  15184, 119010,   9059,
         20309, 118963, 119010,   9059,  20309, 118963,   8887,  49919,   8880,
         11882,  46520,   8964,   9521, 118741,  35506,  24982,  48549,    100,
          9056,  86732,  15184,  14843, 118832,   9279,  11882,   9390,  10459,
          9625, 119438,  10739,   9546, 119245,  16439,  10028,  90537,  12508,
          9056,  86732,  21876,  17342,  14867,   8896, 105197,  10407,   9519,
         14040, 119224,   9390,   9266,  11664,    100,   9706,  17196,   9569,
        118802,  42144, 119322,  11513,   9091,  16985,  12092,   9521, 119008,
         65096,  11018,   8863,   8924,  37388,  12424,   9580, 118762,  10892,
         49307,  28188, 119000,  49307, 118958,  11287,  35866,  49307,  31928,
        119341,  49307,  41605,  21876, 

In [0]:
# 배치 사이즈
batch_size = 4

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [0]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [0]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [0]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [0]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 3

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [0]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [0]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...
  Batch   100  of  1,575.    Elapsed: 0:00:58.
  Batch   200  of  1,575.    Elapsed: 0:01:56.
  Batch   300  of  1,575.    Elapsed: 0:02:54.
  Batch   400  of  1,575.    Elapsed: 0:03:51.
  Batch   500  of  1,575.    Elapsed: 0:04:49.
  Batch   600  of  1,575.    Elapsed: 0:05:47.
  Batch   700  of  1,575.    Elapsed: 0:06:44.
  Batch   800  of  1,575.    Elapsed: 0:07:42.
  Batch   900  of  1,575.    Elapsed: 0:08:40.
  Batch 1,000  of  1,575.    Elapsed: 0:09:37.
  Batch 1,100  of  1,575.    Elapsed: 0:10:35.
  Batch 1,200  of  1,575.    Elapsed: 0:11:33.
  Batch 1,300  of  1,575.    Elapsed: 0:12:30.
  Batch 1,400  of  1,575.    Elapsed: 0:13:28.
  Batch 1,500  of  1,575.    Elapsed: 0:14:26.

  Average training loss: 0.17
  Training epcoh took: 0:15:09

Running Validation...
  Accuracy: 0.99
  Validation took: 0:00:28

======== Epoch 2 / 3 ========
Training...
  Batch   100  of  1,575.    Elapsed: 0:00:58.
  Batch   200  of  1,575.    Elap

In [0]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    750.    Elapsed: 0:00:16.
  Batch   200  of    750.    Elapsed: 0:00:32.
  Batch   300  of    750.    Elapsed: 0:00:48.
  Batch   400  of    750.    Elapsed: 0:01:04.
  Batch   500  of    750.    Elapsed: 0:01:20.
  Batch   600  of    750.    Elapsed: 0:01:37.
  Batch   700  of    750.    Elapsed: 0:01:53.

Accuracy: 0.99
Test took: 0:02:01


In [0]:
model.save_pretrained('/content/gdrive/My Drive/final/bert')
tokenizer.save_pretrained('/content/gdrive/My Drive/final/bert')

# https://notebooks.azure.com/kieran-mcdonald/projects/pytorchclass201905/html/3_PyTorch_bert.ipynb

('/content/gdrive/My Drive/final/bert/vocab.txt',
 '/content/gdrive/My Drive/final/bert/special_tokens_map.json',
 '/content/gdrive/My Drive/final/bert/added_tokens.json')

In [0]:
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 512

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [0]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [0]:
logits = test_sentences(['매운음식이 땡길때  쭈꾸미덮밥 맛있게 만들기  청양고추 대파 매콤한 음식이 땡기는날 쭈꾸미덮밥을 만들어볼게요'])

print(logits)
print(np.argmax(logits))


[[-4.3577275  4.3873816]]
1


In [0]:
logits = test_sentences([data['total'][10]])

print(logits)
print(np.argmax(logits))

[[-3.387054  3.394875]]
1


In [0]:
logits = test_sentences([data['total'][0]])

print(logits)
a=np.argmax(logits)

[[-3.891578  3.81001 ]]


In [0]:
data['total_bert']=data['hot']

In [0]:
for i in range(0,91558):
  print(i)
  if data['hot'][i]==2:
    logits = test_sentences([data['total'][i]])
    temp=np.argmax(logits)
    data['total_bert'][i]=temp

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
86559
86560
86561
86562
86563
86564
86565
86566
86567
86568
86569
86570
86571
86572
86573
86574
86575
86576
86577
86578
86579
86580
86581
86582
86583
86584
86585
86586
86587
86588
86589
86590
86591
86592
86593
86594
86595
86596
86597
86598
86599
86600
86601
86602
86603
86604
86605
86606
86607
86608
86609
86610
86611
86612
86613
86614
86615
86616
86617
86618
86619
86620
86621
86622
86623
86624
86625
86626
86627
86628
86629
86630
86631
86632
86633
86634
86635
86636
86637
86638
86639
86640
86641
86642
86643
86644
86645
86646
86647
86648
86649
86650
86651
86652
86653
86654
86655
86656
86657
86658
86659
86660
86661
86662
86663
86664
86665
86666
86667
86668
86669
86670
86671
86672
86673
86674
86675
86676
86677
86678
86679
86680
86681
86682
86683
86684
86685
86686
86687
86688
86689
86690
86691
86692
86693
86694
86695
86696
86697
86698
86699
86700
86701
86702
86703
86704
86705
86706
86707
86708
86709
86710
86711
86712
86713
86714
86715
86716
86717
86718
8671

In [0]:
data

,id,title,ingredients,description,step,total,hot,bert,total_bert
0,6927971,매운 팽이버섯 조림 간단 반찬술안주맥주안주소주안주 굿굿,팽이버섯 고추장 진간장 설탕 다진마늘 물,밥반찬으로도 술안주로도 좋은 매운 팽이버섯 초림,"['팽이버섯 밑둥을 자르고 한입 크기 덩어리로 분리해줍니다', '팬에 기름을 두르고...",매운 팽이버섯 조림 간단 반찬술안주맥주안주소주안주 굿굿 팽이버섯 고추장 진간장 설탕...,1,0,1
1,6867650,다른 반찬이 필요없는 매운 어묵 비빔밥,사각어묵 땡초 통마늘 고추장 굴소스 올리고당 고춧가루 후추 통깨 올리브유 맛술 참기름,다른 반찬이 필요없는 어묵 양념 비빔밥 재료 착한 가격에 손쉽게 만들어 먹을수 있...,"['어묵 ᆞ땡초 다져주시고 통마늘 듬성듬성 다져주세요', '팬에 위 준비한 재료를 ...",다른 반찬이 필요없는 매운 어묵 비빔밥 사각어묵 땡초 통마늘 고추장 굴소스 올리고당...,1,0,1
2,6860267,간단 자취요리 미친 매운맛 빨간오뎅 집에서 만들어 먹기 Spicy fish cakes,어묵 물 고추 고추장 간장 설탕 다진마늘,안녕하세요 여러분 오늘은 빨간오뎅을 만들어 봤어요 빨간 어묵이 맞지만 상호명이기에 ...,"['재료를 준비합니다어묵 물 고추 고추장 간장 설탕 다진마늘', '고추 1개는 썰고...",간단 자취요리 미친 매운맛 빨간오뎅 집에서 만들어 먹기 Spicy fish cak...,1,0,1
3,6910384,매운맛 쏙 마늘 꿀조림 만들기,간장 꿀 맛술 마늘,마늘은 매운맛때문에 먹기힘들죠 그런데 이렇게 하니 아이들도 먹을수있고 마늘이 꿀맛...,"['재료 마늘2컵 꿀3큰술맛술1큰술조림장 간장2큰술꿀3큰술맛술1큰술', '마늘은 껍...",매운맛 쏙 마늘 꿀조림 만들기 간장 꿀 맛술 마늘 마늘은 매운맛때문에 먹기힘들죠 ...,1,0,1
4,6883669,매운콩나물잡채,당면 콩나물 양파 당근 대파 물 참기름 통깨 고추장 고춧가루 간장 설탕 다진 마늘 ...,콩나물잡채,['당면은 찬물에 밀 불려서 준비해 둡니다대략 30분 이상 불려주심 된답니다당면 전...,매운콩나물잡채 당면 콩나물 양파 당근 대파 물 참기름 통깨 고추장 고춧가루 간장 ...,1,0,1
...,...,...,...,...,...,...,...,...,...
91553,6844474,부드럽고 달콤한 디저트 우유 생크림 젤리 만드는 법,우유 생크림 설탕 젤라틴 분말 초코시럽 딸기시럽 럼주 딸기시럽 초코시럽 물 젤라틴 ...,고소하고 부드러운 우유와 생크림에 쌉쌀한 초코와 상큼한 딸기가 환상의 디저트를 만들...,['냄비에 우유 생크림 설탕 젤라틴 분말을 넣고 중불에서 젤라틴이 녹을 정도로 끓여...,부드럽고 달콤한 디저트 우유 생크림 젤리 만드는 법 우유 생크림 설탕 젤라틴 분말 ...,0,0,0
91554,6844473,오리훈제 야채샐러드,오리훈제슬라이스 쌈배추 파프리카 쌀뜨물 사과 양배추 간장 들깨가루 들기름 레몬즙 올...,야채가 더 많이 들어가서 위에 부담도 적고 오리훈제 야채샐러드 한접시면 양양가득 이...,['오리훈제 야채샐러드 재료 알아봅니다 오리훈제슬라이스 20조각 쌈배추 3장 파프리...,오리훈제 야채샐러드 오리훈제슬라이스 쌈배추 파프리카 쌀뜨물 사과 양배추 간장 들깨가...,0,0,0
91555,6844471,오징어 굴 소스 야채 볶음밥,오징어 콩나물 양배추 파 마늘 양파 당근 브로콜리 파프리카 밥 식용유 들기름 굴소소,딸 친구가 갑자기 놀러 왔어요 밥통에 밥은 있는데 반찬이 영 그래서 냉동실에 손질해...,"['야채 잔뜩 썰어 놨어요 숙주 없어 콩나물 준비했어요', '밥은 2공기 준비하세요...",오징어 굴 소스 야채 볶음밥 오징어 콩나물 양배추 파 마늘 양파 당근 브로콜리 파프...,2,0,1
91556,6844470,깨쟁이요리최고의브런치시금치 바질 페스토시금치 바질 파스타 스파게티,바질페스토 스파게티면 양파 채썬마늘 닭가슴살 생크림.우유 파마산치즈 바질페스토,집에서 직접키운 바질로 만든 바질페스토 시금치를 곁들여 만든 시금치 바질페스토 파스...,['바질페스토시금치 14단바질2팩올리브유2컵잣12컵마늘2알파마산치즈3Ts앤초비1마리...,깨쟁이요리최고의브런치시금치 바질 페스토시금치 바질 파스타 스파게티 바질페스토 스파게...,2,0,0


In [0]:
data.to_csv("/content/gdrive/My Drive/final/bert1.csv", mode='w', index=False)

In [0]:
data1['total_bert']=data['hot']